# Serve GPT-J 6B on SageMaker with DeepSpeed Container

In this notebook, we explore how to host a fine-tuned GPT-J 6B parameter model on SageMaker using the latest container launched using from DeepSpeed and DJL. DJL provides for the serving framework while DeepSpeed is the key sharding library we leverage to enable hosting of large models. We use DJLServing as the model serving solution in this example. DJLServing is a high-performance universal model serving solution powered by the Deep Java Library (DJL) that is programming language agnostic. To learn more about DJL and DJLServing, you can refer to our recent blog post (https://aws.amazon.com/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference/).

Language models have recently exploded in both size and popularity. In 2018, BERT-large entered the scene and, with its 340M parameters and novel transformer architecture, set the standard on NLP task accuracy. Within just a few years, state-of-the-art NLP model size has grown by more than 500x with models such as OpenAI’s 175 billion parameter GPT-3 and similarly sized open source Bloom 176B raising the bar on NLP accuracy. This increase in the number of parameters is driven by the simple and empirically-demonstrated positive relationship between model size and accuracy: more is better. With easy access from models zoos such as Hugging Face and improved accuracy in NLP tasks such as classification and text generation, practitioners are increasingly reaching for these large models. However, deploying them can be a challenge because of their size.

Model parallelism can help deploy large models that would normally be too large for a single GPU. With model parallelism, we partition and distribute a model across multiple GPUs. Each GPU holds a different part of the model, resolving the memory capacity issue for the largest deep learning models with billions of parameters. This notebook uses tensor parallelism techniques which allow GPUs to work simultaneously on the same layer of a model and achieve low latency inference relative to a pipeline parallel solution.

SageMaker has rolled out DeepSpeed container which now provides users with the ability to leverage the managed serving capabilities and help to provide the un-differentiated heavy lifting.

In this notebook, we deploy the fine tuned GPT-J 6B model. DeepSpeed is used for tensor parallelism inference while DJLServing handles inference requests and the distributed workers. For further reading on DeepSpeed you can refer to https://arxiv.org/pdf/2207.00032.pdf 


In [ ]:
# Instal boto3 library to create model and run inference workloads
%pip install -Uqq boto3 awscli

## Section to Download Model from S3

In this section we download the model archive from S3. We will decompress the file and inspect the artifacts. Afterwards, we will copy the binary file back to an S3 location.

The below step to download and then upload to S3 can take several minutes since the model size is extremely large

In [ ]:
import sagemaker

bucket = sagemaker.session.Session().default_bucket()
print(bucket)

Next cell controls which local path to use for fetching the model

In [ ]:
local_model_path = "./model"

<div class="alert alert-block alert-info">
<b>Model Location</b> <br>
Set the model_s3_uri variable to the S3 location of the fine-tuned `model.tar.gz` file.   
It should be of the form `s3://{bucket}/smp-tensorparallel-outputdir/{training-job-name}/output/model.tar.gz`

In [ ]:
model_s3_uri = "s3://sagemaker-us-west-2-232838030412/smp-tensorparallel-outputdir/smp-gpt-j-6B-p4d24x-tp8-pp1-sdp1-bs8-2023-03-24-13-47-11-995/output/model.tar.gz"

In [ ]:
# model_s3_uri = #"<replace-this-with-the-S3-URI>"

The following cell downloads the model from S3 and places it in the local model path we defined above.

In [ ]:
! rm -rf $local_model_path
! mkdir -p $local_model_path || echo "Dir exists"
! aws s3 cp $model_s3_uri $local_model_path

The following commands, extract the binary from the archive, clean up our local directory and rename the model to `gptj.pt`

In [ ]:
! tar -xvf $local_model_path/model.tar.gz fullmodel.pt

In [ ]:
! rm $local_model_path/model.tar.gz
! mv fullmodel.pt $local_model_path/gptj.pt

### Upload to S3 using the awscli 
We will now copy the extracted model to a S3 bucket

In [ ]:
s3_model_prefix = "fine-tuned-gptj-6b/model" 

In [ ]:
! aws s3 cp './model/gptj.pt' s3://{bucket}/{s3_model_prefix}/ 

## Create SageMaker compatible Model artifact and Upload Model to S3

SageMaker needs the model to be in a Tarball format. In this notebook we are going to create the model with the Inference code to shorten the end point creation time. In the Inference code we kick of a multi threaded approach to download the model weights into the container using awscli

The tarball is in the following format

```
code
├──── 
│   └── model.py
│   └── requirements.txt
│   └── serving.properties

```

The actual model is stored in S3 location and will be downloaded into the container directly when the endpoint is created. For that we will pass in two environment variables

1.  "MODEL_S3_BUCKET" Specify the S3 Bucket where the model artifact is
2.  "MODEL_S3_PREFIX" Specify the S3 prefix for where the model artifacts file are actually located

This will be used in the model.py file to read in the actual model artifacts. 

- `model.py` is the key file which will handle any requests for serving. It is also responsible for loading the model from S3
- `requirements.txt` has the awscli library needed to be installed when the container starts up.
- `serving.properties` is the script that will have environment variables which can be used to customize model.py at run time.


In [ ]:
!mkdir -p code_gptj

In [ ]:
%%writefile ./code_gptj/model.py
from djl_python import Input, Output
import os
import deepspeed
import torch
import torch.distributed as dist
import sys
import subprocess
import time
from glob import glob
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from transformers import GPTJModel, GPTJConfig
from transformers.models.opt.modeling_opt import OPTDecoderLayer
from smart_open import open as smart_open
import io

predictor = None


def check_config():
    local_rank = os.getenv("LOCAL_RANK")
    curr_pid = os.getpid()
    print(
        f"__Number CUDA Devices:{torch.cuda.device_count()}:::local_rank={local_rank}::curr_pid={curr_pid}::"
    )

    if not local_rank:
        return False
    return True

def no_init(loading_code):
    def dummy(self):
        return

    modules = [torch.nn.Linear, torch.nn.Embedding, torch.nn.LayerNorm]
    original = {}
    for mod in modules:
        original[mod] = mod.reset_parameters
        mod.reset_parameters = dummy

    result = loading_code()
    for mod in modules:
        mod.reset_parameters = original[mod]

    return result


def get_model():

    if not check_config():
        raise Exception(
            "DJL:DeepSpeed configurations are not default. This code does not support non default configurations"
        )

    deepspeed.init_distributed("nccl")
    tensor_parallel = int(os.getenv("TENSOR_PARALLEL_DEGREE", "2"))
    local_rank = int(os.getenv("LOCAL_RANK", "0"))
    model_dir = "/tmp/model"
    bucket = os.environ.get("MODEL_S3_BUCKET")
    key_prefix = os.environ.get("MODEL_S3_PREFIX")
    curr_pid = os.getpid()
    print(
        f"__Number CUDA Devices:{torch.cuda.device_count()}:tensor_parallel={tensor_parallel}::curr_pid={curr_pid}::"
    )

    print(f"rank: {local_rank} pid={curr_pid}::")
    if local_rank == 0:
        if f"{model_dir}/DONE" not in glob(f"{model_dir}/*"):
            print(f"Starting Model downloading files pid={curr_pid}::")
            try:
                proc_run = subprocess.run(
                    ["aws", "s3", "cp", "--recursive", f"s3://{bucket}/{key_prefix}", model_dir],
                    capture_output=True,
                    text=True,
                )  # python 7 onwards
                print(f"Model download finished pid={curr_pid}::")

                # write file when download complete. Could use dist.barrier() but this makes it easier to check if model is downloaded in case of retry
                with open(f"{model_dir}/DONE", "w") as f:
                    f.write("download_complete")

                print(
                    f"Model download:DONE checkmark written pid={curr_pid}::return_code:{proc_run.returncode}:stderr:{proc_run.stderr}:"
                )
                proc_run.check_returncode()  # to throw the error in case there was one

            except subprocess.CalledProcessError as e:
                print(
                    "Model download failed: Error:\nreturn code: ",
                    e.returncode,
                    "\nOutput: ",
                    e.stderr,
                )
                raise  # FAIL FAST

    dist.barrier()

    print(f"Load the Model pid={curr_pid}::")

    model = no_init(lambda: AutoModelForCausalLM.from_pretrained(
                                        "EleutherAI/gpt-j-6B",
                                        revision='float16',
                                        torch_dtype=torch.float16,
     ))

    load_path = f"s3://{bucket}/{key_prefix}/gptj.pt"
    with smart_open(load_path, 'rb') as f:
        buffer = io.BytesIO(f.read())
        model.load_state_dict(torch.load(buffer))

    tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
    
    model = deepspeed.init_inference(
        model,
        mp_size=tensor_parallel,
        dtype=torch.float16,
        replace_method="auto",
        replace_with_kernel_inject=True,
    )
    generator = pipeline(
        task="text-generation", model=model, tokenizer=tokenizer, device=local_rank
    )
    return generator


def handle(inputs: Input) -> None:
    global predictor
    if not predictor:
        predictor = get_model()

    if inputs.is_empty():
        # Model server makes an empty call to warmup the model on startup
        return None

    data = inputs.get_as_string()
    result = predictor(data, do_sample=True)
    return Output().add(result)

#### Serving.properties has engine parameter which tells the DJL model server to use the DeepSpeed engine to load the model

In [ ]:
%%writefile ./code_gptj/serving.properties
engine = DeepSpeed
# gpu.minWorkers=2
# gpu.maxWorkers=3

#### Requirements will tell the container to load these additional libraries into the container. We need these to download the model into the container

In [ ]:
%%writefile ./code_gptj/requirements.txt
boto3
awscli
smart_open
transformers

In [ ]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path

#### Create required variables and initialize them to create the endpoint, we leverage boto3 for this

In [ ]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_code_prefix = (
    "large-model-djl-gptj6b/code_gptj"  # folder within bucket where code artifact will go
)

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

**Image URI for the DJL container is being used here**

In [ ]:
# inference_image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/djl-ds:latest"
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.19.0-deepspeed0.7.3-cu113"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

**Create the Tarball and then upload to S3 location**

In [ ]:
!rm model.tar.gz
!tar czvf model.tar.gz code_gptj

In [ ]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

In [ ]:
print(f"S3 Model Prefix where the model files are -- > {s3_model_prefix}")
print(f"S3 Model Bucket is -- > {model_bucket}")

### To create the end point the steps are:

1. Create the Model using the Image container and the Model Tarball uploaded earlier
2. Create the endpoint config using the following key parameters

    a) Instance Type is ml.g4dn.4xlarge
    
    b) ModelDataDownloadTimeoutInSeconds is 2400 which is needed to ensure the Model downloads from S3 successfully,
    
    c) ContainerStartupHealthCheckTimeoutInSeconds is 2400 to ensure health check starts after the model is ready
    
3. Create the end point using the endpoint config created    
    

One of the key parameters here is **TENSOR_PARALLEL_DEGREE** which essentially tells the DeepSpeed library to partition the models along 8 GPU's. This is a tunable and configurable parameter. For the purpose of this notebook we would like to leave these as **default settings**.

This parameter also controls the no of workers per model which will be started up when DJL serving runs. As an example if we have a 8 GPU machine and we are creating 8 partitions then we will have 1 worker per model to serve the requests. For further reading on DeepSpeedyou can follow the link https://www.deepspeed.ai/tutorials/inference-tutorial/#initializing-for-inference. 

In [ ]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"gptj6b-djl-ds")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact,
        "Environment": {
            "MODEL_S3_BUCKET": bucket,
            "MODEL_S3_PREFIX": s3_model_prefix,
            "TENSOR_PARALLEL_DEGREE": "1",
        },
    },
    # Uncomment if providing networking configs
    # VpcConfig=privateVpcConfig
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

VolumnSizeInGB has been left as commented out. You should use this value for Instance types which support EBS volume mounts. The current instance we are using comes with a pre configured space and does not support additional volume mounts

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.4xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 200
            "ModelDataDownloadTimeoutInSeconds": 1200,
            "ContainerStartupHealthCheckTimeoutInSeconds": 1200,
        },
    ],
)
endpoint_config_response

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

#### Wait for the end point to be created.
However while that happens, let us look at the critical areas of the helper files we are using to load the model
1. We will look at the code snippets for model.py to see the model downloading mechanism
2. Requirements.txt to see the required libraries to be loaded
3. Serving.properties to see the environment related properties

In [ ]:
# This is the code snippet which is responsible to load the model from S3
! sed -n '97,121p' code_gptj/model.py

In [ ]:
# This is the code snippet which loads the libraries into the container needed for run
! sed -n '1,3p' code_gptj/requirements.txt

In [ ]:
# This is the code snippet which shows the environment variables being used to customize runtime
! sed -n '1,3p' code_gptj/serving.properties

### This step can take ~ 15 min or longer so please be patient

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

#### Leverage the Boto3 to invoke the endpoint. 

This is a generative model so we pass in a Text as a prompt and Model will complete the sentence and return the results


In [ ]:
%%time
smr_client.invoke_endpoint(
    EndpointName=endpoint_name, Body="Amazon.com is the best", ContentType="text/plain"
)["Body"].read().decode("utf8")

## Conclusion
In this post, we demonstrated how to use SageMaker large model inference containers to host two large language models, BLOOM-176B and OPT-30B. We used DeepSpeed’s model parallel techniques with multiple GPUs on a single SageMaker machine learning instance. For more details about Amazon SageMaker and its large model inference capabilities, refer to the following:

* Amazon SageMaker now supports deploying large models through configurable volume size and timeout quotas (https://aws.amazon.com/about-aws/whats-new/2022/09/amazon-sagemaker-deploying-large-models-volume-size-timeout-quotas/)
* Real-time inference – Amazon SageMake (https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html)



## Clean Up

In [ ]:
# - Delete the end point
sm_client.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
# - In case the end point failed we still want to delete the model
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

#### Optionally delete the model checkpoint from S3

In [ ]:
!aws s3 rm --recursive s3://{bucket}/{s3_model_prefix}

In [ ]:
s3_client = boto3.client("s3")

In [ ]:
len(s3_client.list_objects(Bucket=bucket, Prefix=f"{s3_model_prefix}/")["Contents"])